In [ ]:
#installing necessary libraries
%pip install torch torchvision torchaudio

   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/109.3 MB 11.2 MB/s eta 0:00:10
   -- ------------------------------------- 6.3/109.3 MB 19.3 MB/s eta 0:00:06
   --- ------------------------------------ 10.5/109.3 MB 19.3 MB/s eta 0:00:06
   ----- ---------------------------------- 15.7/109.3 MB 21.1 MB/s eta 0:00:05
   ------- -------------------------------- 20.4/109.3 MB 21.2 MB/s eta 0:00:05
   --------- ------------------------------ 25.4/109.3 MB 21.8 MB/s eta 0:00:04
   ----------- ---------------------------- 30.7/109.3 MB 22.1 MB/s eta 0:00:04
   ------------ --------------------------- 35.4/109.3 MB 22.5 MB/s eta 0:00:04
   -------------- ------------------------- 40.6/109.3 MB 22.7 MB/s eta 0:00:04
   ---------------- ----------------------- 45.9/109.3 MB 22.8 MB/s eta 0:00:03
   ------------------ --------------------- 50.6/109.3 MB 22.8 MB/s eta 0:00:03
   -------------------- ------------------- 55.6/10

In [4]:
%pip install torchmetrics

   ---------------------------------------- 0.0/983.2 kB ? eta -:--:--
   ---------------------------------------- 983.2/983.2 kB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#importing necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchmetrics.classification import Accuracy   
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim

In [ ]:
#Loading the dataset
df= pd.read_csv(r'C:\Users\LENOVO\Downloads\IDS-2018_Intrusion.csv')
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [10]:
df.shape

(1048575, 80)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  float64
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

In [11]:
#Data Preprocessing
#Checking for missing values
df.isnull().sum()

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 80, dtype: int64

In [12]:
#checking for duplicate values
df.duplicated().sum()

5459